In [63]:
import argparse
import gzip
import json
import logging
import os
import pickle
import random
import tarfile
from datetime import datetime
import numpy as np
import torch.cuda
import tqdm
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, losses, InputExample, evaluation
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from accelerate import Accelerator
from accelerate import DistributedDataParallelKwargs

In [64]:
model_name="EleutherAI/gpt-neo-1.3B"
cache_folder="/data/private/huxiaomeng/pretrained_models/"
max_seq_length=300

In [65]:
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length,cache_folder=cache_folder)

Exception ignored in: <function tqdm.__del__ at 0x7f294cb44a70>
Traceback (most recent call last):
  File "/home/huxiaomeng/anaconda3/envs/sgpt/lib/python3.7/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/home/huxiaomeng/anaconda3/envs/sgpt/lib/python3.7/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'
Some weights of GPTNeoModel were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['transformer.h.1.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.17.attn.attention.bias', 'transformer.h.23.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.13.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.3.attn.attention

Using Maximum Sequence Length:  300


In [99]:
word_embedding_model.tokenizer.pad_token = word_embedding_model.tokenizer.eos_token

In [68]:
tokens = ["[SOS]", "{SOS}"]
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=True)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

# Will be replaced with the rep tokens in the model ones
# The problem is we don't know if a text is query or document when tokenizing in the Transformer.py module, 
# so we use the SOS tokens as an identifier if we have a query or document at hand & then replace them
# If we would directly use the brackets here, they may become part of another token
word_embedding_model.bos_spec_token_q = word_embedding_model.tokenizer.encode("[SOS]", add_special_tokens=False)[0]
word_embedding_model.bos_spec_token_d = word_embedding_model.tokenizer.encode("{SOS}", add_special_tokens=False)[0]

word_embedding_model.bos_spec_token_q_rep = word_embedding_model.tokenizer.encode("[", add_special_tokens=False)[0]
word_embedding_model.eos_spec_token_q = word_embedding_model.tokenizer.encode("]", add_special_tokens=False)[0]

word_embedding_model.bos_spec_token_d_rep = word_embedding_model.tokenizer.encode("{", add_special_tokens=False)[0]
word_embedding_model.eos_spec_token_d = word_embedding_model.tokenizer.encode("}", add_special_tokens=False)[0]

word_embedding_model.replace_bos = True

In [73]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), "weightedmean")
model = SentenceTransformer(cache_folder=cache_folder,modules=[word_embedding_model, pooling_model])

In [82]:
word_embedding_model.tokenizer.encode("[SOS]", add_special_tokens=False)[0]

50257

In [83]:
word_embedding_model.tokenizer.encode("[", add_special_tokens=False)[0]

58

In [38]:
model = SentenceTransformer(model_name_or_path=model_name,cache_folder=cache_folder)

Using Maximum Sequence Length:  300


In [2]:

a=[1,2,3]
b=[6,7,8]

In [3]:
for i,j in zip(a,b):
    i+=1
    j+=1
    

In [5]:
b

[6, 7, 8]

In [75]:
print(model._first_module().eos_spec_token_d)

92


In [137]:
 import torch
 a=torch.tensor([-1,5,-9,-10,52,50])

In [138]:
top_k_values, top_k_idx = torch.topk(a, 3, dim=0, largest=True, sorted=False)

In [143]:
a[top_k_idx[0]]

tensor(52)

In [147]:
top_k_values.tolist()

[52, 50, 5]

In [119]:
top_k_idx

tensor([[4, 5, 3],
        [4, 5, 1]])

In [126]:
top_k_idx

tensor([[4, 5, 3],
        [4, 5, 1]])

In [113]:
sorted([1,9,-1],reverse=True)

[9, 1, -1]

In [111]:
a

tensor([1, 2, 3, 6, 9, 7])

In [26]:

for name, param in model.named_parameters():
    print(name)
    if "bias" in name:
        # Freeze all except bias
        continue 
    param.requires_grad = False

0.auto_model.wte.weight
0.auto_model.wpe.weight
0.auto_model.h.0.ln_1.weight
0.auto_model.h.0.ln_1.bias
0.auto_model.h.0.attn.attention.k_proj.weight
0.auto_model.h.0.attn.attention.v_proj.weight
0.auto_model.h.0.attn.attention.q_proj.weight
0.auto_model.h.0.attn.attention.out_proj.weight
0.auto_model.h.0.attn.attention.out_proj.bias
0.auto_model.h.0.ln_2.weight
0.auto_model.h.0.ln_2.bias
0.auto_model.h.0.mlp.c_fc.weight
0.auto_model.h.0.mlp.c_fc.bias
0.auto_model.h.0.mlp.c_proj.weight
0.auto_model.h.0.mlp.c_proj.bias
0.auto_model.h.1.ln_1.weight
0.auto_model.h.1.ln_1.bias
0.auto_model.h.1.attn.attention.k_proj.weight
0.auto_model.h.1.attn.attention.v_proj.weight
0.auto_model.h.1.attn.attention.q_proj.weight
0.auto_model.h.1.attn.attention.out_proj.weight
0.auto_model.h.1.attn.attention.out_proj.bias
0.auto_model.h.1.ln_2.weight
0.auto_model.h.1.ln_2.bias
0.auto_model.h.1.mlp.c_fc.weight
0.auto_model.h.1.mlp.c_fc.bias
0.auto_model.h.1.mlp.c_proj.weight
0.auto_model.h.1.mlp.c_proj.bias


In [97]:
text=["[SOS] I love you","[SOS] Do you love me"
]

In [95]:
a=model._first_module()

In [81]:
a.bos_spec_token_q
a.bos_spec_token_q_rep

58

In [89]:
a=[1,2,3]
b=[4,5,6]
out={'a':a,'b':b}
newa=[]
newb=[]
for i,j in zip(out['a'],out['b']):
    print(i,j)
    i=5
    j=5
    newa.append(i)
    newb.append(j)
out['a']=newa
out['b']=newb

1 4
2 5
3 6


In [91]:
out['b']

[5, 5, 5]

In [98]:
model.tokenize(text)

{'input_ids': tensor([[   58,   314,  1842,   345,    60, 50256],
         [   58,  2141,   345,  1842,   502,    60]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1]])}

In [59]:
tokens = ["[SOS]", "{SOS}"]
model._first_module().tokenizer.add_tokens(tokens, special_tokens=True)

0

In [62]:
model._first_module().tokenizer.encode("{SOS}",add_soecial_tokens=False)[0]

TypeError: _batch_encode_plus() got an unexpected keyword argument 'add_soecial_tokens'

In [60]:
model._first_module().tokenizer

PreTrainedTokenizerFast(name_or_path='/data/private/huxiaomeng/sgpt/checkpoints/sgpt-1.3G/15599/', vocab_size=50257, model_max_len=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'})